# If using Google Colab

In [ ]:
!git clone https://github.com/jorisbc/EEIST_complexity

In [ ]:
!ls EEIST_complexity

# Main

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

# Read and explore data

In [ ]:
df = pd.read_csv('country_product_export_2019.zip', index_col=None)
df.head()

In [ ]:
pivot = df.pivot_table(columns=??, index=??, values=??, aggfunc=np.mean, fill_value=0)
pivot.tail()

In [ ]:
pivot[pivot.index.str.contains('Cocoa', case=False)]

In [ ]:
# make a bar plot showing the top 15 exports of one of the cacao products
cacao_product_export_sorted = pivot.loc[??, :].sort_values(ascending=??).head(??)

cacao_product_export_sorted.plot(kind=??, figsize=(10, 3))

# Create M matrix

$$\text{RCA}_{cp} = \frac{x_{cp} / x_p}{x_c / x}$$

$$ M_{cp} = \text{RCA}_{cp} > 1$$

In [ ]:
x_product = pivot.??
x_country = pivot.??
x_total = pivot.??.??

In [ ]:
rca = (pivot / x_product).div(x_country / x_total, axis=0)

In [ ]:
rca['Netherlands'].sort_values(ascending=False).head(8).plot(kind='barh', figsize=(5, 8))

In [ ]:
# The M matrix is a matrix of 1s and 0s, where Mij = 1 if country i exports product j with RCA > 1, and 0 otherwise.
M = np.heaviside(??)

## Create product space adjacency matrix

between i and j at time t: 
$$ \phi_{i,j} = \min\left[ P(RCAx_{i} | RCAx{j}), P(RCAx_{j} | RCAx{i}) \right]$$

, where, empirically:

$$ P(RCAx_{i} | RCAx_{j}) = \frac{\# (RCAx_{i} \& RCAx_{j})}{\# RCA_{j}} $$

So, using M, we have

$$P(RCAx_{p} | RCAx_{p'})  = \frac{\sum_c M_{cp} * M_{cp'} }{ \sum_c M_{cp'} }$$

In [ ]:
P_RCA = M.dot(M.transpose()) / np.diag(M.dot(M.transpose()))

# make symmetric by taking the minimum of the two values
phi = pd.concat([P_RCA, P_RCA.transpose()]).groupby(level=0).min()

In [ ]:
# make list
phi_list = phi.stack()

# sort by value
phi_list.??(ascending=False, inplace=True)

# remove diagonal
phi_list = phi_list[phi_list.index.get_level_values(0) != phi_list.index.get_level_values(1)]

# rename multiindex and make dataframe
phi_list.index.names = ['product1', 'product2']
phi_list.name = 'phi'
phi_list = phi_list.to_frame()

In [ ]:
pd.set_option('display.max_colwidth', 700)
phi_list.head(5)

In [ ]:
display(phi_list[75:100])

# Create 'product spacee' network using networkx packagk

In [ ]:
# Phi can be used to create a network of products: the product space
G = nx.from_pandas_edgelist(phi_list.reset_index()[:10000], source=??, target=??, edge_attr=??)

In [ ]:
# calculate minimum spanning tree (keep everything connected but only via the strongest connections)

# set phi to -phi
for u, v, d in G.edges(data=True):
    d['phi'] = -d['phi']

# get maximum spanning tree
T = nx.maximum_spanning_tree(G, weight='phi')

# set phi to -phi
for u, v, d in T.edges(data=True):
    d['phi'] = -d['phi']

In [ ]:
# draw 'product space' network
plt.figure(figsize=(10, 10))
pos = nx.spring_layout(T, k=0.1, weight=??)

edge_width = [1*d['phi'] for (u, v, d) in T.edges(data=True)]

nx.draw(T, pos, node_size=10, width=??, edge_color='grey', node_color='grey', with_labels=False)

# Country - product density

$$ d^c_j = \frac{\sum_i M_{ci} \phi_{ij}}{\sum_i \phi_{ij}} $$

In [ ]:
country = ?? # choose a country

In [ ]:
competitive_products = M[M[country] == ?].index

In [ ]:
# all the products that country ?? is 'competitive' in
competitive_products

In [ ]:
proximity_to_country = pd.DataFrame(index=phi.index, columns=['proximity'])

In [ ]:
# it is still empty now:
proximity_to_country

In [ ]:
# help function for next step
def calc_proximity(prod, competitive_products, phi):
    proximity = phi.loc[prod, competitive_products].sum() / phi.loc[??, :].sum()
    return proximity

In [ ]:
# loop over each product and calculate proximity to country of that product

for product in proximity_to_country.index:
    proximity_to_country.loc[product, 'proximity'] = calc_proximity(??, ??, phi)

In [ ]:
# display the top and bottom 5 products in terms of proximity to country
proximity_to_country.sort_values(by='proximity', ascending=False, inplace=True)

# visualise proximity with RCA and PCI

In [ ]:
# if google colab
import EEIST_complexity.complexity as complexity

In [ ]:
# if on local machine
import complexity

In [ ]:
# calculate the ECI and PCI
M, eci, eci_list, pci, pci_list = complexity.pivot_to_eci_pci(pivot)

In [ ]:
# reset index
proximity_to_country = proximity_to_country.loc[pivot.index]

In [ ]:
# add PCI to proximity_to_country
proximity_to_country['pci'] = -pci # why -pci and not pci?

In [ ]:
proximity_to_country.sort_values('pci', ascending=False)

In [ ]:
# add rca to proximity_to_country
proximity_to_country['rca'] = rca[??]

In [ ]:
proximity_to_country.sort_values('rca', ascending=False)

In [ ]:
import plotly.express as px

In [ ]:
# interactive plot with names on hover
fig = px.scatter(proximity_to_country, x=??, y=??, hover_name=proximity_to_country.index,
                 title = 'Proximity to ' + country + ' vs. Product Complexity Index (PCI)')
fig.show()

In [ ]:
# interactive plot with names on hover
rca_threshold = 1
proximity_to_c_thresholded = proximity_to_country[proximity_to_country.rca < rca_threshold]
fig = px.scatter(proximity_to_c_thresholded, x=??, y=??, hover_name=proximity_to_c_thresholded.index,
                    title = 'Proximity to ' + country + ' vs. Product Complexity Index (PCI) for products with RCA < 1')

fig.show()

Compare with https://green-transition-navigator.org/

# Ubiquity and diversity

In [ ]:
ubiquity = M.sum(axis=1).sort_values(ascending=False)
diversity = M.sum(axis=0).sort_values(ascending=False)

In [ ]:
# top and bottom countries in terms of diversity
diversity

In [ ]:
# top and bottom products in terms of ubiquity
ubiquity

In [ ]:
plt.imshow(M.loc[ubiquity.index, diversity.index], interpolation='none', aspect='auto')

In [ ]:
# why does this look like a triangle?

# ECI and PCI calculation
ECI is the eigenvector associated with the second largest eigenvalue of the matrix 
$$\tilde{M} = D^{-1} M U^{-1} M^T $$

$$\tilde{M} = D^{-1} S$$

$$S_{cc'} = \sum_p \frac{M_{cp} M_{c'p}}{u_c}$$

Symmetricaly, PCI is the eigenvector associated with the second largest eigenvalue of the matrix 
$$\hat{M} = U^{-1} M^T D^{-1} M $$

In [ ]:
def M_to_Mhat(M):
    # k_c = country . k_p = product. D = diversity, U = ubiquity

    k_c = M.sum(axis=0) # diversity
    k_p = M.sum(axis=1) # ubiquity
    D = np.diag(k_c)
    U = np.diag(k_p)

    S_tilde = (M.T).dot(np.linalg.inv(U)).dot(M)
    S_hat = (M).dot(np.linalg.inv(D)).dot(M.T)

    M_tilde = np.linalg.inv(D).dot(S_tilde)
    M_hat = np.linalg.inv(U).dot(S_hat)

    return M_hat, M_tilde


def Mtilde_to_complexity(M_tilde, M, type='eci'):
    # the PCI is the eigenvector associated with the second largest right eigenvalue of M_hat: M_hat * eci = lambda*eci
    # from the docs: a   eigvec[:,i] = eigval[i]        b   eigvec[:,i]
    eigenValues, eigenVectors = linalg.eig(M_tilde)

    idx = eigenValues.argsort()[::-1]   
    eigenValues = eigenValues[idx]
    eigenVectors = eigenVectors[:,idx]

    # take second largest eigenvector
    pci = np.real(eigenVectors[:, 1])
    
    idx = pci.argsort()[::-1]
    if type == 'eci':
        pci_list = M.dropna(axis=1).columns[idx]
    elif type == 'pci':
        pci_list = M.dropna().index[idx]

    return pci, pci_list